# Use LP based Adaptive Shadow Ban Policy



In [ ]:
#!pip install control

In [62]:
%%time
import numpy as np
import control.optimal as obc
import control as ct
from pyoptsparse import  Optimization, OPT
import pyoptsparse as pos
import scipy.sparse as sparse
from scipy.sparse import coo_matrix, csr_matrix, diags, identity

from scipy.optimize import linprog

import matplotlib.pyplot as plt
import scripts.shadowban_pyoptsparse as sb
from scripts.shadowban_adaptive import *
from scripts.opinion_visualization import *
from scripts.load_data import * # load Twitter network
from scripts.class_dynamics import * # class for opinion dynamics simulation
import networkx as nx
import scipy.sparse as sp # save coo_matrix A and E in .npz
import pickle # save ndarray rates and opinions0 in .pkl

import pandas as pd
import time
import seaborn as sns
from datetime import datetime


CPU times: user 181 µs, sys: 831 µs, total: 1.01 ms
Wall time: 1.03 ms


In [ ]:
%reload your_package_name #reload script

# Initialization

## Load Network

For your network `G` you need the following node features
   1. `rates` = array of posting rate of each node
   2. `opinions0` = array of initial opinions of each node

### Load Synthetic Network

In [ ]:
n0 = 20
nc = int(n0/2)
sizes = [nc, nc]
p0, p1 = 8/n0, 1/n0
probs = [[p0, p1], [p1, p0]]
G = nx.DiGraph(nx.stochastic_block_model(sizes, probs, seed=0))

for i in range(nc):
    rate = 1
    opinion0 = i/(nc-1)/2 
    opinion1 = 1-i/(nc-1)/2 
    G.nodes[i]['opinion'] = opinion0
    G.nodes[i+nc]['opinion'] = opinion1
    G.nodes[i]['rate'] = 1
    G.nodes[i+nc]['rate'] = 1
fig = draw_network(G)

A = nx.adjacency_matrix(G)
A = A.tocoo()
assert n0 == A.shape[0]  #number of nodes in network should equal shape of A
rates = np.array([G.nodes[v]["rate"] for v in G.nodes()])  #posting rates of nodes
opinions0 = np.array([G.nodes[v]["opinion"] for v in G.nodes()])  #initial opinions of nodes

plt.hist(opinions0)
plt.show()


#adjacency matrix of network
# A = nx.adjacency_matrix(G)
# A = A.tocoo()
E0 = nx.incidence_matrix(G,oriented=True)
E0 = E0.tocoo()
ind = E0.data>0
E = coo_matrix((E0.data[ind], (E0.row[ind], E0.col[ind])), E0.shape)  #incidence matrix with only tail of edge

network_params = {'A':A, 'rates':rates, 'opinions0':opinions0, 'E':E}

### Load Twitter Network

#### Specify network_name

In [65]:
%%time
# "Brexit_sample_02_agent"
# "GiletsJaunes_sample_02_agent" 
# "2016_Second_Presidential_Debate_random_sample" "2016_Second_Presidential_Debate_full"
network_name = "data/2016_Second_Presidential_Debate_full"

print(f"Number of nodes = {G.number_of_nodes()}")
print(f"Number of edges = {G.number_of_edges()}")

Number of nodes = 77167
Number of edges = 5021347
CPU times: user 30.1 ms, sys: 34 ms, total: 64.1 ms
Wall time: 69.5 ms


#### Save coo_matrix A,E in .npz and others in .pkl (optional)

In [ ]:
G, network_params = G_to_params(network_name + ".csv")

# Save the adjacency matrix A
sp.save_npz(network_name + '_A.npz', network_params['A'])

# Save the incidence matrix E
sp.save_npz(network_name + '_E.npz', network_params['E'])

# Save rates
with open(network_name + '_rates.pkl', 'wb') as file:
    pickle.dump(network_params['rates'], file)

# Save opinions0
with open(network_name + '_opinions0.pkl', 'wb') as file:
    pickle.dump(network_params['opinions0'], file)

#### Reload network_params

In [69]:
# Load the adjacency matrix A
A = sp.load_npz(network_name + '_A.npz')

# Load the incidence matrix E
E = sp.load_npz(network_name + '_E.npz')

# Load rates
with open(network_name + '_rates.pkl', 'rb') as file:
    rates = pickle.load(file)

# Load opinions0
with open(network_name + '_opinions0.pkl', 'rb') as file:
    opinions0 = pickle.load(file)

network_params = {'A':A, 'E':E, 'rates':rates, 'opinions0':opinions0}
G = nx.DiGraph(network_params['A'])

network_params

{'A': <77167x77167 sparse matrix of type '<class 'numpy.int64'>'
 	with 5021347 stored elements in COOrdinate format>,
 'E': <77167x5021347 sparse matrix of type '<class 'numpy.float64'>'
 	with 5021347 stored elements in COOrdinate format>,
 'rates': array([0.43333333, 0.2       , 0.13333333, ..., 0.13333333, 0.03333333,
        2.33333333]),
 'opinions0': array([0.59144 , 0.57384 , 0.58773 , ..., 0.52017 , 0.965356, 0.58052 ])}

## Set Simulation and Optimization Parameters

In [7]:
#simulation parameters
tau = 0.1
omega = 0.1

Tf = 10     #final time of simulator
npts_eval = 100  #number of time discretization points for simulation
npts = 3
assert npts < npts_eval

#optimization parameters 
smax = 0.5  #max strength of shadow banning at any time
OBJECTIVE = 'MEAN'  #integral cost type

assert OBJECTIVE in ['MEAN','VARMIN','VARMAX']

## Create params dictionary for problem

In [8]:
nv = G.number_of_nodes()
ne = G.number_of_edges()

#names of edges for shadowbanning in format uij
input_names = []
for e in G.edges():
    input_names.append(f"u{e[0]}{e[1]}")
#names of nodes opinions in format thetai
output_names = []
for v in G.nodes():
    output_names.append(f"theta{v}")
    
assert nv == A.shape[0]  #number of nodes in network should equal shape of A
assert ne == E.shape[1]  #number of edges in network should equal columns of E


#parameters of network model
# params = {'A':A,'E':E,'rates':rates,'tau':tau,'omega':omega, 'opinions0':opinions0,
#          'npts_eval':npts_eval, 'Tf':Tf,'smax':smax,
#           'OBJECTIVE':OBJECTIVE, 'npts':npts}
additional_params = {'tau': tau, 'omega': omega, 'npts_eval': npts_eval, 'Tf': Tf, 'smax': smax, 'OBJECTIVE': OBJECTIVE, 'npts': npts}

params = {**network_params, **additional_params} #parameters of network model
params

{'A': <77167x77167 sparse matrix of type '<class 'numpy.int64'>'
 	with 5021347 stored elements in COOrdinate format>,
 'rates': array([0.43333333, 0.2       , 0.13333333, ..., 0.13333333, 0.03333333,
        2.33333333]),
 'opinions0': array([0.59144 , 0.57384 , 0.58773 , ..., 0.52017 , 0.965356, 0.58052 ]),
 'tau': 0.1,
 'omega': 0.1,
 'npts_eval': 100,
 'Tf': 10,
 'smax': 0.5,
 'OBJECTIVE': 'MEAN',
 'npts': 3,
 'E': <77167x5021347 sparse matrix of type '<class 'numpy.float64'>'
 	with 5021347 stored elements in COOrdinate format>}

# Define Input/Output Systems

In [9]:
sys_no_agent = ct.NonlinearIOSystem(
        updfcn =sb.sys_update, outfcn= None, states=nv,
        inputs=ne, outputs = nv,
        name='shadowban network no agent', params=params)

sys_lp = ct.NonlinearIOSystem(
        updfcn =sys_update_lp, outfcn= None, states=nv,
        inputs=ne, outputs = nv,
        name='shadowban network lp', params=params)

In [ ]:
# ct.optimal.state_range_constraint(sys_no_agent, np.zeros(nv), np.ones(nv))

# Opinions with no Shadow Banning

In [10]:
%%time
T, Opinions_no_agent, _ = sb.simulate_opinions(params, sys_no_agent)
obj_no_agent = sb.cost_sim(OBJECTIVE, Opinions_no_agent)

KeyboardInterrupt: 

# Opinions with LP Shadow Banning

In [ ]:
%%time

for OBJECTIVE in ['MEAN','VARMIN','VARMAX']:
    params['OBJECTIVE'] = OBJECTIVE
    obj_no_agent = sb.cost_sim(OBJECTIVE, Opinions_no_agent)
    #LP shadow banning 
    T, Opinions_lp, _ = sb.simulate_opinions(params, sys_lp)
    obj_lp = sb.cost_sim(OBJECTIVE, Opinions_lp)

    fig = plt.figure(figsize = (10,4))
    plt.subplot(1,2,1)
    plot_opinion_quantiles(T, Opinions_no_agent)
#     plot_opinions(T, Opinions_no_agent)
    plt.ylim(0, 1)
    plt.title(f"{nv} nodes {OBJECTIVE} = {obj_no_agent:.3f}")

    plt.subplot(1,2,2)
    plot_opinion_quantiles(T, Opinions_lp)
#     plot_opinions(T, Opinions_lp)
    plt.ylim(0, 1)
    plt.title(f"{nv} nodes LP Shadow Ban {OBJECTIVE} = {obj_lp:.3f}")
    
    #plt.savefig(f"figures/shadowban_{OBJECTIVE}_{nv}_node_sbm.jpeg")

    plt.show()